In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-summarization-vietnews-with-simplet5/__results__.html
/kaggle/input/text-summarization-vietnews-with-simplet5/__notebook__.ipynb
/kaggle/input/text-summarization-vietnews-with-simplet5/__output__.json
/kaggle/input/text-summarization-vietnews-with-simplet5/custom.css
/kaggle/input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-6-train-loss-1.2434-val-loss-1.1818/config.json
/kaggle/input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-6-train-loss-1.2434-val-loss-1.1818/spiece.model
/kaggle/input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-6-train-loss-1.2434-val-loss-1.1818/tokenizer.json
/kaggle/input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-6-train-loss-1.2434-val-loss-1.1818/tokenizer_config.json
/kaggle/input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-6-train-loss-1.2434-val-loss-1.1818/pytorch_model.bin
/kaggle/input/text-summarization-vietnews-with-simplet5/ou

In [2]:
test_df = pd.read_csv('/kaggle/input/vietnew-csv/vietnew_dataset/test_tokenized')

In [3]:
test_metric = test_df.copy()
test_metric = test_metric.sample(frac=0.2, replace=True, random_state=1)

In [4]:
!pip install --upgrade simplet5

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 28.6 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=8549e992ee446d183e899df69f36e0dc7e62e984d7d38642f25ea28ac7d82baf
  Stored in directory: /root/.cache/pip/wheels/30/1b/96/47ad5fda68def6635961fd0ca312fc0ce3670936ae09d97c43
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
  Attempting uninstall: pyDeprecate
    Found existing installation: pyDeprecate 0.3.2
    Uninstalling pyDeprecate-0.3.2:
      Successfully uninstalled pyDeprecate-0.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4

In [5]:
!pip install rouge

In [6]:
from simplet5 import SimpleT5
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="NlpHUST/t5-small-vi-summarization")

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [7]:
model.load_model("t5","../input/text-summarization-vietnews-with-simplet5/outputs/simplet5-epoch-7-train-loss-1.2205-val-loss-1.167", use_gpu=True)
sumarize = test_metric['target_text'].iloc[2]
# predict
model.predict(sumarize)

#analy_train['Number_of_words'] = analy_train.source_text.apply(lambda x: len(x.split()))
test_metric['summarize_text']= test_metric.target_text.apply(lambda x: model.predict(x))
test_metric.to_csv('metrics.csv', index=False)

In [8]:
from rouge import Rouge 
df = pd.read_csv('metrics.csv')
rouge = Rouge()
scores = rouge.get_scores(df['target_text'], df['summarize_text'],avg=True)

In [9]:
print(pd.DataFrame(data =scores))

    rouge-1   rouge-2   rouge-l
r  0.843277  0.768511  0.832423
p  0.711860  0.637138  0.703439
f  0.765011  0.688749  0.755627
